## Cliff Walking
### Environment Build

In [29]:
import copy

class CliffWalkingEnv:
    """
    Cliff Walking Environment
    """
    def __init__(self, ncol=12, nrow=4):
        self.ncol = ncol # 定义网格世界的列
        self.nrow = nrow # 定义网格世界的行
        # 转移矩阵 P[state][action] = [(p, next_state, reward, done)] 包含下一个状态和奖励
        self.P = self.createP()

    def createP(self):
        # 初始化
        P = [[[] for j in range(4)] for i in range(self.ncol * self.nrow)]
        # 4种动作，change[0]:上，change[1]：下，change[2]:左，change[3]：右。坐标原点（0,0）
        # 定义在左上角
        change = [[0, -1], [0, 1], [-1, 0], [1, 0]]
        for i in range(self.nrow):
            for j in range(self.ncol):
                for a in range(4):
                    # 位置在悬崖或者目标状态，因为无法继续交互，任何动作奖励都为0
                    if i == self.nrow -1 and j > 0:
                        P[i*self.ncol + j][a] = [(1, i*self.ncol + j, 0, True)]
                        continue
                    # 其他位置
                    next_x = min(self.ncol - 1, max(0, j + change[a][0]))
                    next_y = min(self.nrow - 1, max(0, i + change[a][1]))
                    next_state = next_y * self.ncol + next_x
                    reward = -1
                    done = False
                    # 下一个位置在终点或者是悬崖
                    if next_y == self.nrow - 1 and next_x > 0:
                        done = True
                        if next_x != self.ncol - 1:
                            reward = -100
                    P[i * self.ncol + j][a] = [(1, next_state, reward, done)]
        return P

### Policy Iteration

In [30]:
class PolicyIteration:
    """
    Algorithm: Policy Iteration
    """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow # 初始化价值为 0 
        self.pi = [[0.25, 0.25, 0.25, 0.25] for i in range(self.env.ncol * self.env.nrow)] # 初始化为均匀随机策略
        self.theta = theta # 策略评估收敛阈值
        self.gamma = gamma # 折扣因子

    def policy_evaluation(self): # 策略评估
        cnt = 1 # 计数器
        while 1:
            max_diff = 0
            new_v = [0] * self.env.ncol * self.env.nrow
            for s in range(self.env.ncol * self.env.nrow):
                qsa_list = [] # 开始计算状态 s 下的所有 Q(s,a)价值
                for a in range(4):
                    qsa=0
                    for res in self.env.P[s][a]:
                        p, next_state, r, done = res
                        qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                    qsa_list.append(self.pi[s][a] * qsa)
                new_v[s] = sum(qsa_list) # 状态价值函数和动作价值函数之间的关系
                max_diff = max(max_diff, abs(new_v[s] - self.v[s]))
            self.v = new_v
            if max_diff < self.theta:
                break
            cnt += 1
        print("策论评估进行 %d 轮后完成"% cnt)
        print(self.v)
    
    def policy_improvement(self): # 策略提升
        for s in range(self.env.nrow * self.env.ncol):
            qsa_list = []
            for a in range(4):
                qsa = 0
                for res in self.env.P[s][a]:
                    p, next_state, r, done = res
                    qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                qsa_list.append(qsa)
            maxq = max(qsa_list)
            cntq = qsa_list.count(maxq) # 计算有几个动作得到了最大的 Q 值
            # 让这些动作均分概率
            self.pi[s] = [1/cntq if q == maxq else 0 for q in qsa_list]
        print("策略提升完成")
        return self.pi
    
    def policy_iteration(self): # 策略迭代
        while 1:
            self.policy_evaluation()
            old_pi = copy.deepcopy(self.pi) # 将列表进行深拷贝，方便接下来进行比较
            new_pi = self.policy_improvement()
            if old_pi == new_pi:
                break


def print_agent(agent, action_meaning, disaster=[], end=[]):
    """
    help fuction:
    打印当前策略在每个状态下的价值以及智能体采取的动作。
    """
    print("状态价值：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            print('%6.6s' % ('%.3f' % agent.v[i*agent.env.ncol + j]), end=' ')
        print()
    
    print("策略：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            if (i*agent.env.ncol+ j) in disaster:
                print('****', end=" ")
            elif (i*agent.env.ncol + j) in end:
                print("EEEE", end=" ")
            else:
                a = agent.pi[i*agent.env.ncol + j]
                pi_str = ""
                for k in range(len(action_meaning)):
                    pi_str += action_meaning[k] if a[k] > 0 else 'o'
                print(pi_str, end=" ")
        print()

env = CliffWalkingEnv()
action_meaning = ['^','v',"<",'>']
theta = 0.001
gamma = 0.9
agent = PolicyIteration(env,theta,gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, list(range(37,47)), [47])

策论评估进行 60 轮后完成
[-27.238559676619346, -28.510206246429455, -29.62885270244113, -30.305628122512452, -30.63180044115316, -30.71290352575707, -30.576247758044943, -30.147026390044594, -29.224052962725274, -27.478407211415796, -24.65095855211554, -21.452783239853783, -33.6318737150897, -36.893230087142776, -38.79777435746743, -39.68392452942374, -40.0493986760689, -40.13912693311535, -40.01645050280487, -39.597576201305145, -38.59325411177045, -36.330852853611376, -31.53565514170566, -23.347152140868065, -47.269744178720174, -58.588300742368254, -61.78662689369652, -62.77814358941198, -63.10031193173464, -63.175112719696, -63.09561179906289, -62.790108249415496, -61.93065649443827, -59.42067040315814, -51.38701026633216, -22.98702501042819, -66.15535964969234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
策略提升完成
策论评估进行 72 轮后完成
[-10.008749065266038, -10.008749065266038, -10.008749065266038, -10.008749065266038, -10.008749065266038, -10.005812617174692, -10.005812617174692, -10.0058

### Value Iteration

In [31]:
class ValueIteration:
    """
    Algorithm: Value Iteration
    """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow # 初始化价值为 0
        self.theta = theta # 价值收敛阈值
        self.gamma = gamma # 折扣因子
        self.pi = [None for i in range(self.env.ncol * self.env.nrow)] # 价值迭代结束后得到的策略

    def value_iteration(self):
        cnt = 0
        while 1:
            max_diff = 0
            new_v = [0] * self.env.ncol * self.env.nrow
            for s in range(self.env.ncol * self.env.nrow):
                qsa_list = [] # 开始计算状态 s 下的所有 Q(s,a)价值
                for a in range(4):
                    qsa = 0
                    for res in self.env.P[s][a]:
                        p, next_state, r, done = res
                        qsa += p*(r + self.gamma * self.v[next_state]* (1 - done))
                    qsa_list.append(qsa)
                new_v[s] = max(qsa_list)
                max_diff = max(max_diff, abs(new_v[s]-self.v[s]))
            self.v = new_v
            if max_diff < self.theta:
                break
            cnt += 1
        print("价值迭代一共进行 %d 轮" % cnt)
        self.get_policy()
    
    # 根据价值函数导出一个贪婪策略
    def get_policy(self):
        for s in range(self.env.ncol * self.env.nrow):
            qsa_list = []
            for a in range(4):
                qsa = 0
                for res in self.env.P[s][a]:
                    p, next_state, r, done = res
                    qsa += p*(r + self.gamma * self.v[next_state]* (1 - done))
                qsa_list.append(qsa)
            maxq = max(qsa_list)
            cntq = qsa_list.count(maxq)
            self.pi[s] = [1/cntq if q == maxq else 0 for q in qsa_list]
                        
env = CliffWalkingEnv()
action_meaning = ['^','v',"<",'>']
theta = 0.001
gamma = 0.9
agent = ValueIteration(env,theta,gamma)
agent.value_iteration()
print_agent(agent, action_meaning, list(range(37,47)), [47])

价值迭代一共进行 14 轮
[-7.7123207545039, -7.458134171671, -7.175704635190001, -6.861894039100001, -6.5132155990000005, -6.12579511, -5.6953279000000006, -5.217031, -4.68559, -4.0951, -3.439, -2.71, -7.458134171671, -7.175704635190001, -6.861894039100001, -6.5132155990000005, -6.12579511, -5.6953279000000006, -5.217031, -4.68559, -4.0951, -3.439, -2.71, -1.9, -7.175704635190001, -6.861894039100001, -6.5132155990000005, -6.12579511, -5.6953279000000006, -5.217031, -4.68559, -4.0951, -3.439, -2.71, -1.9, -1.0, -7.458134171671, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ov

## Frozen Lake
### Environment Build

In [32]:
import gym
env = gym.make("FrozenLake-v1") # 创建环境
env = env.unwrapped # 解封装才能访问状态转移矩阵
env.render() # 环境渲染

In [33]:
holes = set()
ends = set()
for s in env.P:
    for a in env.P[s]:
        for s_ in env.P[s][a]:
            if s_[2] == 1.0: # 获得奖励为1，代表是目标
                ends.add(s_[1])
            if s_[3] == True:
                holes.add(s_[1])
holes = holes - ends
print("冰洞的索引：", holes)
print("目标的索引：", ends)

冰洞的索引： {11, 12, 5, 7}
目标的索引： {15}


In [34]:
for a in env.P[14]: # 查看目标左边一格的状态转移信息
    print(env.P[14][a])

[(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False)]
[(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)]
[(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]
[(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False)]


### Policy Iteration

In [35]:
action_meaning = ['<',"v",">","^"]
theta = 1e-5
gamma = 0.9
agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, [5,7,11,12], [15])

策论评估进行 25 轮后完成
[0.00444902630020078, 0.004202515082223095, 0.010049027274031766, 0.0041046187710215044, 0.006705089770241343, 0.0, 0.026326459459953325, 0.0, 0.01866569208458738, 0.05760016156109084, 0.10696591783020536, 0.0, 0.0, 0.13037747186484136, 0.3914848363371555, 0.0]
策略提升完成
策论评估进行 58 轮后完成
[0.06882358543345177, 0.06135752557874739, 0.07436816193032114, 0.05576157533078255, 0.09179332520777392, 0.0, 0.11218582405176862, 0.0, 0.14538679668310456, 0.2474635485038229, 0.29959407556204687, 0.0, 0.0, 0.3799117892450865, 0.6390075188838532, 0.0]
策略提升完成
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


### Value Iteration

In [36]:
action_meaning = ['<',"v",">","^"]
theta = 1e-5
gamma = 0.9
agent = ValueIteration(env, theta, gamma)
agent.value_iteration()
print_agent(agent, action_meaning, [5,7,11,12], [15])

价值迭代一共进行 60 轮
[0.06882785168244751, 0.06136114051594982, 0.07437079805287419, 0.05576447370854035, 0.09179720475646835, 0.0, 0.11218724255070185, 0.0, 0.1453899377624117, 0.24746566611902698, 0.2995955663221989, 0.0, 0.0, 0.3799133179330759, 0.639008319658704, 0.0]
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


## Grid Word(4-1)

In [37]:
class GridWord:
    def __init__(self, ncol = 4, nrow =4):
        self.ncol = ncol
        self.nrow = nrow
        self.P = self.createP()

    def createP(self):
        # 初始化
        P = [[[] for j in range(4)] for i in range(self.ncol * self.nrow)]
        # 4种动作，change[0]:上，change[1]：下，change[2]:左，change[3]：右。坐标原点（0,0）
        # 定义在左上角
        change = [[0, -1], [0, 1], [-1, 0], [1, 0]]
        for i in range(self.nrow):
            for j in range(self.ncol):
                for a in range(4):
                    if i!=0 or j!= 0:
                    # 其他位置
                        next_x = j + change[a][0]
                        next_y = i + change[a][1]
                        if next_x<0 or next_x>self.ncol-1:
                            next_x = j
                        if next_y<0 or next_y>self.nrow-1:
                            next_y = i
                        next_state = next_y * self.ncol + next_x
                        reward = -1
                        done = False
                        P[i * self.ncol + j][a] = [(1, next_state, reward, done)]
                        # 位置在目标状态，因为无法继续交互，任何动作奖励都为0
                    P[self.ncol * self.nrow -1][a] = [(1, self.ncol * self.nrow -1, 0, True)]
                    P[0][a] = [(1, 0, 0, True)]
        return P

In [38]:
env = GridWord()
action_meaning = ['^','v',"<",'>']
theta = 0.001
gamma = 1
agent = PolicyIteration(env,theta,gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, [], [0, 15])

策论评估进行 131 轮后完成
[0.0, -13.989457715062407, -19.98437823213207, -21.982518317902986, -13.989457715062407, -17.986238146361153, -19.98448273123845, -19.98437823213207, -19.984378232132066, -19.98448273123845, -17.986238146361153, -13.989457715062407, -21.982518317902983, -19.984378232132066, -13.989457715062407, 0.0]
策略提升完成
策论评估进行 4 轮后完成
[0.0, -1.0, -2.0, -3.0, -1.0, -2.0, -3.0, -2.0, -2.0, -3.0, -2.0, -1.0, -3.0, -2.0, -1.0, 0.0]
策略提升完成
策论评估进行 1 轮后完成
[0.0, -1.0, -2.0, -3.0, -1.0, -2.0, -3.0, -2.0, -2.0, -3.0, -2.0, -1.0, -3.0, -2.0, -1.0, 0.0]
策略提升完成
状态价值：
 0.000 -1.000 -2.000 -3.000 
-1.000 -2.000 -3.000 -2.000 
-2.000 -3.000 -2.000 -1.000 
-3.000 -2.000 -1.000  0.000 
策略：
EEEE oo<o oo<o ov<o 
^ooo ^o<o ^v<> ovoo 
^ooo ^v<> ovo> ovoo 
^oo> ooo> ooo> EEEE 


In [39]:
env = Ex4_1()
action_meaning = ['^','v',"<",'>']
theta = 0.001
gamma = 1
agent = ValueIteration(env,theta,gamma)
agent.value_iteration()
print_agent(agent, action_meaning, [], [0, 15])

价值迭代一共进行 3 轮
[0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1, 0]
状态价值：
 0.000 -1.000 -2.000 -3.000 
-1.000 -2.000 -3.000 -2.000 
-2.000 -3.000 -2.000 -1.000 
-3.000 -2.000 -1.000  0.000 
策略：
EEEE oo<o oo<o ov<o 
^ooo ^o<o ^v<> ovoo 
^ooo ^v<> ovo> ovoo 
^oo> ooo> ooo> EEEE 
